# Define all constants corresponding to game data

Define resources

In [16]:
RESOURCES = [
    "VP", # Victory points
    "Q", # Quest cards (as rewards to be obtained)
    # "I", # Intrigue cards (as rewards to be obtained) # TODO (later versions): Uncomment this
    "Purple", # Wizard item, i.e. purple cubes
    "White", # Cleric item, i.e. white cubes
    "Black", # Rogue item, i.e. black cubes
    "Orange", # Fighter item, i.e. orange cubes
    "Gold", # Gold
]

Define all quests in the game

In [17]:
QUEST_TYPES = ["Arcana", "Piety", "Skullduggery", "Warfare", "Commerce"]

In [18]:
from collections import namedtuple
Quest = namedtuple("Quest", "name type requirements rewards")

QUESTS = [
    Quest("Simple Warfare", "Warfare", {"Orange": 2}, {"VP": 4}),
    Quest("Simple Skullduggery", "Skullduggery", {"Black": 2}, {"VP": 4}),
    Quest("Simple Piety", "Piety", {"White": 1}, {"VP": 4}),
    Quest("Simple Arcana", "Arcana", {"Purple": 1}, {"VP": 4}),
    Quest("Simple Commerce", "Commerce", {"Gold": 4}, {"VP": 4}),
]

Verify correctness of quest attributes

In [19]:
for quest in QUESTS:
    for rewardKey in quest.rewards:
        assert rewardKey in RESOURCES, quest
    for requirementKey in quest.requirements:
        assert requirementKey in RESOURCES, quest
    assert quest.type in QUEST_TYPES

Demonstrate how to access quest data

In [20]:
orangeQuest = QUESTS[0]
print(orangeQuest)
print(orangeQuest.type)
print(orangeQuest.rewards)
print(orangeQuest.rewards["VP"])

Quest(name='Simple Warfare', type='Warfare', requirements={'Orange': 2}, rewards={'VP': 4})
Warfare
{'VP': 4}
4


Demonstrate that quests are immutable

In [21]:
orangeQuest.rewards = {"black": 2} # Results in AttributeError

AttributeError: can't set attribute

Define number of agents per player as a function of number of players

In [22]:
def agentsPerPlayer(numPlayers: int):
    '''
    Return the number of agents per player
    as a function of the number of players in 
    the game.
    
    Args:
        numPlayers: the total number of players in the game.
        
    Returns:
        the number of agents per player.
    '''
    if numPlayers == 2: return 4
    if numPlayers == 3: return 3
    if numPlayers == 4: return 2
    if numPlayers == 5: return 2 
    else: 
        raise ValueError("Number of players must be an integer between 2 and 5, inclusive.")

Define the Lord cards (i.e. secret identities)

In [23]:
# TODO (later version): uncomment below
LORD_CARDS = []
for i,type1 in enumerate(QUEST_TYPES):
    for type2 in QUEST_TYPES[i+1:]:
        LORD_CARDS.append((type1, type2))
# LORD_CARDS.append("Buildings")

# Create classes for running the game

In [24]:
from random import shuffle

## Player class

In [54]:
class Player():
    def __init__(self, name: str, numAgents: int, 
                 lordCard: tuple[str]) -> None:
        '''
        Initialize the player's name, resources, agents, and VPs.
        
        Args:
            name: the player's name
            numAgents: the number of starting agents for the player
            lord: the lord card (i.e. secret identity) given to the player 
        '''
        self.name = name 
        self.lordCard = lordCard

        self.resources = {
            "Purple": 0,
            "White": 0,
            "Black": 0,
            "Orange": 0,
            "Gold": 0,
            "VP": 0
        }
        for resource in self.resources:
            assert resource in RESOURCES

        self.activeQuests = []
        self.completedQuests = []
        # self.plotQuests = [] # Completed plot quests
        # self.intrigues = []
        self.agents = numAgents
        self.maxAgents = numAgents

        # The score used for training RL agents
        # should NOT be VP alone, but 
        # score = VP + #(agents) + #(gold)//2
        # (maybe include quests/intrigue too?)
        # (if we are really trying to teach 
        # strategy, maybe #(white,purple) 
        # + #(black,orange)/2 + #(gold)/4 
        # to correspond to turn-value instead 
        # of VP-value at endgame? )

    def getQuest(self, quest: Quest):
        '''
        Receive a quest.

        Args: 
            quest: the quest to receive.
        '''
        self.activeQuests.append(quest)

    # TODO (Later version): uncomment this
    # def getIntrigue(self, intrigue: Intrigue):
        # '''
        # Receive an intrigue card.

        # Args: 
        #     intrigue: the intrigue card to receive.
        # '''
    #     self.intrigues.append(intrigue)

    def getResource(self, resource: str, number: int):
        '''
        Receive some number of resources of the same type.

        Args: 
            resource: the type of resource.
            number: the number of that resource type received.
        '''
        if resource not in RESOURCES:
            raise ValueError("Invalid resource type.")
        if resource in ["Q", "I"]:
            raise ValueError("Cannot receive quests or intrigue cards with this function. \
                             Use 'receiveQuest' or 'receive Intrigue', respectively.")
        if number <= 0:
            raise ValueError("Cannot receive nonnegative resource count.")
        
        self.resources[resource] += number

    def getAgent(self):
        '''Receive an additional agent (for future use).'''
        self.maxAgents += 1
        self.agents += 1

    def returnAgents(self):
        '''Return all of this player's agents.'''
        self.agents = self.maxAgents
        
    def completeQuest(self, quest: Quest):
        # Check if the quest can be completed
        validCompletion = True
        for resource in quest.requirements:
            if quest.requirements[resource] > self.resources[resource]:
                validCompletion = False
                
        if validCompletion:
            for resource,number in quest.requirements.items():
                self.resources[resource] -= number
            for resource,number in quest.rewards.items():
                if resource not in RESOURCES:
                    raise ValueError("Invalid resource type.")
                elif resource == "Q":
                    # TODO: Implement this. somehow access gamestate 
                    raise Exception("Not yet implemented.")
                elif resource == "I":
                    # TODO (later version): Implement this
                    raise Exception("This is impossible! Intrigue cards do not exist yet!")
                else:
                    self.resources[resource] += number
        else:
            raise ValueError("Do not have enough resources to complete this quest.")
        
        # Check that all resource counts are still nonnegative
        for resourceNumber in self.resources.values():
            assert resourceNumber >= 0
        

## Game state class

In [55]:
class GameState():
    def __init__(self) -> None:
        '''
        Initialize the game state. Creates the quest stack
        and initializes all buildings states.
        '''
        # Create the quest stack
        self.questStack = QUESTS.copy()
        shuffle(self.questStack)

        # Initialize all building states
        # TODO (in later versions): have all 3 Cliffwatch Inn spots instead of just 1
        # TODO (in later versions): uncomment the commented ones below
        self.purpleOccupied = False 
        self.buildingStates = { # True if the building is occupied
            "Purple": False, # Blackstaff Tower (for Wizards)
            "Orange": False, # Field of Triumph (for Fighters)
            "White": False, # The Plinth (for Clerics)
            "Black": False, # The Grinning Lion Tavern (for Rogues)
            "Gold": False, # Aurora's Realms Shop (for Gold)
            "Quest": False, # Cliffwatch Inn (for Quests)
            # "Castle": False # Castle Waterdeep (for Castle + Intrigue)
            # "Builder": False # Builder's Hall (for buying Buildings)
            # "Waterdeep1": False # Waterdeep Harbor, first slot (for playing Intrigue)
            # "Waterdeep2": False # Waterdeep Harbor, second slot (for playing Intrigue)
            # "Waterdeep3": False # Waterdeep Harbor, third slot (for playing Intrigue)
        }

    def clearBuildings(self):
        '''Clears all buildings to their unoccupied states.'''
        for building in self.buildingStates:
            self.buildingStates[building] = False
    
    def drawQuest(self) -> Quest:
        '''
        Draw the top quest from the quest stack,
        removing it from the stack in the process.
        
        Returns: 
            The top quest from the quest stack.
        '''
        return self.questStack.pop()
    
    def printQuestStack(self) -> None:
        '''Debug function for printing the quest stack.'''
        print("Quest stack (top first):")
        questStackCopy = self.questStack.copy()
        for i in range(len(self.questStack)):
            print(i+1, questStackCopy.pop())

    def displayGameState(self) -> None:
        '''Display the state of the game.'''
        # TODO: Implement this to display the state of the game
        # Will start textually, should be graphically later
        pass

Test the quest stack

In [56]:
gameState = GameState()

for i in range(2):
    print("Drew quest:", gameState.drawQuest())
    gameState.printQuestStack()
    print()

Drew quest: Quest(name='Simple Warfare', type='Warfare', requirements={'Orange': 2}, rewards={'VP': 4})
Quest stack (top first):
1 Quest(name='Simple Piety', type='Piety', requirements={'White': 1}, rewards={'VP': 4})
2 Quest(name='Simple Skullduggery', type='Skullduggery', requirements={'Black': 2}, rewards={'VP': 4})
3 Quest(name='Simple Commerce', type='Commerce', requirements={'Gold': 4}, rewards={'VP': 4})
4 Quest(name='Simple Arcana', type='Arcana', requirements={'Purple': 1}, rewards={'VP': 4})

Drew quest: Quest(name='Simple Piety', type='Piety', requirements={'White': 1}, rewards={'VP': 4})
Quest stack (top first):
1 Quest(name='Simple Skullduggery', type='Skullduggery', requirements={'Black': 2}, rewards={'VP': 4})
2 Quest(name='Simple Commerce', type='Commerce', requirements={'Gold': 4}, rewards={'VP': 4})
3 Quest(name='Simple Arcana', type='Arcana', requirements={'Purple': 1}, rewards={'VP': 4})



## Game interface

Class to control the flow of the game, focused on turn progression and move 
options. Broadly, this class handles anything involving the game state and
the players, while the previous classes handle either only the game state
or only the players.

**Any interactions with player or game states should always involve methods of those classes.**

In [57]:
class GameControl():
    '''
    Class to control the flow of the game, 
    focused on turn progression and move 
    options. 
    '''
    def __init__(self, numPlayers: int = 3, numRounds: int = 8, 
                 playerNames = None):
        '''
        Initialize the game state and players.

        Args: 
            numPlayers: the number of players in the game
            numRounds: the number of rounds in the game
            playerNames (optional): the names for each player
        '''
        # Initialize the remaining number of rounds
        self.roundsLeft = numRounds

        # Initialize the GameState
        self.gameState = GameState()

        # Check that we have a valid number of players
        assert numPlayers >= 2 and numPlayers <= 5
        self.numPlayers = numPlayers

        # Set default player names
        self.playerNames = playerNames
        if playerNames == None:
            self.playerNames = [
                "PlayerOne", "PlayerTwo", 
                "PlayerThree", "PlayerFour",
                "PlayerFive"
            ][:numPlayers]
        
        # Shuffle the lord cards
        shuffled_lord_cards = LORD_CARDS.copy()
        print(shuffled_lord_cards[:5])
        shuffle(shuffled_lord_cards)
        print(shuffled_lord_cards[:5])

        # Initialize the players
        self.players = []
        for i in range(numPlayers):
            print(shuffled_lord_cards[i])
            self.players.append(Player(self.playerNames[i], agentsPerPlayer(numPlayers),
                                       shuffled_lord_cards[i]))

        # Deal quest cards to players
        for _ in range(2):
            for player in self.players:
                player.getQuest(self.gameState.drawQuest())

        # TODO (later version): Deal intrigue cards to players

        # Define the turn order 
        shuffled_indices = list(range(1,numPlayers+1))
        shuffle(shuffled_indices)
        self.turnOrder = {shuffled_indices[i]: player for i,player in enumerate(self.players)}

        # Set the current place in the turn order (1,...,numPlayers)
        self.currentTurn = 1

        # Finally, start a new round (at this stage, just 
        # decrements roundsLeft and places VPs on
        # buildings at builder's hall)
        self.newRound()

    def newRound(self):
        '''Reset the board at the beginning of each round.'''
        self.roundsLeft -= 1
        # TODO (later version): put VPs on buildings at bulider's hall

        # Reset all buildings
        self.gameState.clearBuildings()

        # TODO (later version): put new resources on buildings that need them

        # Get new agent at fifth round
        if self.roundsLeft == 4:
            for player in self.players:
                player.getAgent()

        # Return all agents
        for player in self.players:
            player.returnAgents()

    def takeTurn(self):
        '''Take a single turn in the turn order.'''
        # TODO: Implement 
        currentPlayer = self.turnOrder[self.currentTurn]
        possibleMoves = [] # Fill this 
        move = currentPlayer.selectMove(possibleMoves) # Implement this
        # Execute this move. Maybe have a different function for 
        # selecting a building to play at vs other sub-selections
        # such as a quest to complete or an intrigue card to play?
        self.turnOrder += 1

    def runGame(self):
        '''Umbrella function to run the game.'''
        while self.roundsLeft > 0:
            while self.turnOrder <= self.numPlayers:
                self.takeTurn()
            self.newRound()
        

In [68]:
gameControl = GameControl(numPlayers=2)
print(gameControl.playerNames)
for i,player in gameControl.turnOrder.items():
    print(i,player.name, player.lordCard)

[('Arcana', 'Piety'), ('Arcana', 'Skullduggery'), ('Arcana', 'Warfare'), ('Arcana', 'Commerce'), ('Piety', 'Skullduggery')]
[('Arcana', 'Skullduggery'), ('Piety', 'Warfare'), ('Skullduggery', 'Commerce'), ('Piety', 'Commerce'), ('Skullduggery', 'Warfare')]
('Arcana', 'Skullduggery')
('Piety', 'Warfare')
['PlayerOne', 'PlayerTwo']
1 PlayerOne ('Arcana', 'Skullduggery')
2 PlayerTwo ('Piety', 'Warfare')


# RL Model 

In [ ]:
def score(resources):
    '''Compute an RL agent's score.'''
    # See comment in Player.__init__ about score
    # Maybe include quests/intrigues?
    score = 0.
    for resource,number in resources.items():
        if resource in ["Purple", "White", "VP"]:
            score += number 
        elif resource in ["Orange", "Black"]:
            score += number / 2.
        elif resource == "Gold":
            score += number / 4.
        else:
            raise ValueError("Invalid resource type.")
    return score 